# PESTPP-IES - History Matching For "Nothing", Uncertainty Analysis for Free

It is very common to want to restart PESTPP-IES using existing results.  The most common reason is to evaluate the prior parameter ensemble - that is, to run the prior through the model and check the simulated results for coherence and common sense.  Once you are happy with (or not completely unhappy with) the prior results, you might then want to try a small number of iterations with PESTPP-IES.  If your model runs fast, then you can just fire off PESTPP-IES again just as before but with `noptmax` greater than 0.  But if your model takes a while, it can be costly to re-run the prior parameter ensemble again.  In this case, you can easily restart PESTPP-IES with the existing ensemble files.  In this notebook, we will demonstrate this workflow.  We note that while we will focus on restarting with the prior ensemble results, PESTPP-IES can be restarted from any previous PESTPP-IES result files


### Admin

The modified Freyberg model is introduced in another tutorial notebook (see "freyberg intro to model"). The current notebook picks up following the "freyberg observation and weights" notebook, in which a high-dimensional PEST dataset was constructed using `pyemu.PstFrom`. You may also wish to go through the "intro to pyemu" notebook beforehand.

The next couple of cells load necessary dependencies and call a convenience function to prepare the PEST dataset folder for you. Simply press `shift+enter` to run the cells.

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd


In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template')
# get the previously generated PEST dataset
org_t_d = os.path.join("..","part2_02_obs_and_weights","freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb' notebook")
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

## Preparing for PESTPP-IES

Just as before:

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')
pst = pyemu.Pst(pst_path)

In [ ]:
# check to see if obs&weights notebook has been run
if not pst.observation_data.observed.sum()>0:
    raise Exception("You need to run the '/part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb' notebook")

In [ ]:
pst.pestpp_options

In [ ]:
pst.pestpp_options["ies_num_reals"] = 50
pst.pestpp_options['ies_parameter_ensemble'] = 'prior_pe.jcb'
pst.control_data.noptmax = -1 #run the prior parameter ensemble and quit
pst.pestpp_options["ies_bad_phi_sigma"] = 2.0 #middle ground value
pst.write(os.path.join(t_d, 'freyberg_mf6.pst'))

In [ ]:
psutil.cpu_count(logical=False)

In [ ]:
num_workers = 8 #update this according to your resources

In [ ]:
m_d = os.path.join('master_ies_prior')

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'freyberg_mf6.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

### How to restart PESTPP-IES

PESTPP-IES can be restarted easily by supplying the following files for a previous PESTPP-IES run:
 - parameter ensemble (iteration specific), option name: `ies_parameter_ensemble`
 - obs+noise ensemble, option name `ies_observation_ensemble`
 - simulated output ensemble (iteration specific), option name: `ies_restart_observation_ensemble`
 
The parameter ensemble and simulated output ensembles must be from the same iteration but PESTPP-IES has no way of knowing this, so its up to you not to mess this up!  In the current example, we want to restart with the prior ensemble results, so the files we seek are "<case_name>.0.par.csv" and "<case>.0.obs.csv" for the parameter and simulated ensemble files respectively (note that if you are using binary formats, the files would end with ".jcb" instead of ".csv").  
    
The obs+noise ensemble is written during the initialization PESTPP-IES and does not change over a PESTPP-IES run. Its name is always "<case_name>.obs+noise.csv" (or ".jcb").  
    
So!  To restart PESTPP-IES, we just need to add/modify these 3 options aaaaannnndddd WHAMMY! you jump straight to upgrade calculations...
    
...not so fast - these files exist in the master directory (in this example, "master_ies_prior" and we need to get them into the template directory - #pathing.  And there is a very important point here: 
    
#### IF YOU DONT CHANGE THE NAME OF THE RESTARTING FILES OR THE PEST CONTROL FILE, THEN THE RESTARTING FILES WILL BE OVERWRITTEN THE NEXT TIME YOU RUN PESTPP-IES!!!  
    
The best practice here is to just rename the restarting files for safe keeping...

In [ ]:
for filename,argname in zip(["freyberg_mf6.0.par.csv","freyberg_mf6.0.obs.csv","freyberg_mf6.obs+noise.csv"],
                            ["ies_parameter_ensemble","ies_restart_observation_ensemble","ies_observation_ensemble"]):

    # a simple renaming scheme
    renamed_filename = "restart_"+filename
    # copy the original restart file from the prior master dir to the renamed filename in the template dir
    shutil.copy2(os.path.join(m_d,filename),os.path.join(t_d,renamed_filename))
    #modify/set the pestpp option
    pst.pestpp_options[argname] = renamed_filename

In [ ]:
pst.pestpp_options

Ok, should be G2G...

In [ ]:
pst.write(os.path.join(t_d,"freyberg_mf6.pst"),version=2)
pyemu.os_utils.run("pestpp-ies freyberg_mf6.pst",cwd=t_d)

Huh?!  What happened there?  

Well since we didnt change `noptmax` from -1, PESTPP-IES just initialized the same as it ever would: it loaded the existing ensemble files, made an initial phi report and quit.  Actually this is a simple and efficient check to see that all the mechanics are setup as you intended.  Ok, for reals now:

In [ ]:
pst.control_data.noptmax = 3
pst.write(os.path.join(t_d,"freyberg_mf6.pst"),version=2)

In [ ]:
m_d = os.path.join('master_ies_restart')

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'freyberg_mf6.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )